In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import vgg16, VGG16_Weights

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [2]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Define transforms for preprocessing and data augmentation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # Defining the Normalizing values as stated in torchvision.models documentation
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Using device: cuda


In [3]:

# Load dataset
#\\wsl.localhost\Ubuntu-22.04\home\student\DL\inaturalist_12K
dataset_path = "inaturalist_12K/"
train_data = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform)
test_data = datasets.ImageFolder(root=f"{dataset_path}/val", transform=transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_subset, val_subset = random_split(train_data, [train_size, val_size])

# Define DataLoaders
batch_size = 64

# Loading data onto ram
class PreloadDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.data = [(img, label) for img, label in dataset]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_subset = PreloadDataset(train_subset)
val_subset = PreloadDataset(val_subset)
test_data = PreloadDataset(test_data)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Training set size: {len(train_subset)}")
print(f"Validation set size: {len(val_subset)}")
print(f"Test set size: {len(test_data)}")

Training set size: 7999
Validation set size: 2000
Test set size: 2000


In [4]:
model = vgg16(weights=VGG16_Weights.DEFAULT)

# Freezing all layers
for p in model.parameters():
    p.requires_grad = False

#print(model)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
for p in model.classifier[6].parameters():
    p.requires_grad = True

print(model)
print()
model = model.to(device)

'''for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)'''

print(f"Model is on: {next(model.parameters()).device}")

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/student/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|████████████████████████████████████████████████████████████████████████████████| 528M/528M [00:06<00:00, 81.6MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
best_val_loss = float('inf')

for epoch in range(epochs):
    print(f"\nEpoch [{epoch+1}/{epochs}]")

    # Training Phase
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")


Epoch [1/10]
Train Loss: 0.8926 | Train Acc: 70.21% | Val Loss: 0.7168 | Val Acc: 75.40%

Epoch [2/10]
Train Loss: 0.6858 | Train Acc: 76.76% | Val Loss: 0.6985 | Val Acc: 77.65%

Epoch [3/10]
Train Loss: 0.6360 | Train Acc: 78.52% | Val Loss: 0.6973 | Val Acc: 76.70%

Epoch [4/10]
Train Loss: 0.6035 | Train Acc: 79.62% | Val Loss: 0.6751 | Val Acc: 77.30%

Epoch [5/10]
Train Loss: 0.6016 | Train Acc: 79.86% | Val Loss: 0.6771 | Val Acc: 77.45%

Epoch [6/10]
Train Loss: 0.5621 | Train Acc: 81.02% | Val Loss: 0.6810 | Val Acc: 78.00%

Epoch [7/10]
Train Loss: 0.5437 | Train Acc: 81.75% | Val Loss: 0.6672 | Val Acc: 78.35%

Epoch [8/10]
Train Loss: 0.5527 | Train Acc: 81.32% | Val Loss: 0.6755 | Val Acc: 77.95%

Epoch [9/10]
Train Loss: 0.5433 | Train Acc: 81.37% | Val Loss: 0.6816 | Val Acc: 78.00%

Epoch [10/10]
Train Loss: 0.5233 | Train Acc: 81.84% | Val Loss: 0.6699 | Val Acc: 78.95%


In [6]:
# Testing the Best Model
model.eval()
test_loss, correct, total = 0.0, 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    avg_test_loss = test_loss / len(test_loader)

print(f"Test Loss: {avg_test_loss:.4f} | Test Acc: {test_accuracy:.2f}%")

Test Loss: 0.6105 | Test Acc: 80.45%
